In [1]:
import glob
import numpy as np
import pandas as pd
import datetime
import xarray as xr
import dask
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import pickle
def myround(x, base=5):
    return base * round(x/base)
def date(x):
    return x.date()



In [ ]:
era_filepath = '/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/reanalysis_data/ERA5/regression_vars/'
dtypes = {'latitude':np.float32, 'longitude':np.float32, 'r_eff':np.float32, 'npoints':np.float32, 'LWP':np.float32, 'COT':np.float32,
                       'CTH':np.float32, 'signal':np.float32,  'particle':np.int32, 'latitude_1':np.float32, 'longitude_1':np.float32,
                       'r_eff_1':np.float32, 'LWP_1':np.float32, 'COT_1':np.float32, 'CTH_1':np.float32, 'npoints_1':np.float32, 'latitude_3':np.float32,
                       'longitude_3':np.float32, 'r_eff_3':np.float32, 'LWP_3':np.float32, 'COT_3':np.float32, 'CTH_3':np.float32, 'npoints_3':np.float32,
                       'terra':np.bool_, 'ocean':np.bool_,'Nd':np.float32, 'Nd_1':np.float32 , 'Nd_3':np.float32,  'hours_diff':np.int32, 'chil':np.bool_, 'azor':np.bool_, 'cver':np.bool_, 'ango':np.bool_}
colu = ['longitude',  'latitude', 'ocean','overpass' ]

for year in ['2014','2015','2016','2017','2018','2019','2021',][0:1]:
    files = glob.glob('/gws/nopw/j04/eo_shared_data_vol1/satellite/modistracks/{}/inc_EIS_par_??'.format(year))
    h5 = pd.read_parquet(files,
                     columns=colu,
                     # dtype=dtypes,
                     # blocksize=200e6
                        )
    
    for ran_path in glob.glob(era_filepath + '*{}*.nc'.format(year))[0:1]:
        era = xr.open_dataset(ran_path)
        era_int = era.interp(time=xr.DataArray(h5.index, dims='obs'), longitude=xr.DataArray(h5.longitude, dims='obs'), latitude=xr.DataArray(h5.latitude, dims='obs'))
        era_int = era_int.to_dataframe()
        era_int['latitude'] = era_int.latitude.apply(myround, )
        era_int['longitude'] = era_int.longitude.apply(myround, )
        era_int['time'] = era_int.time.apply(date)
        era_int = era_int.groupby([era_int['latitude'], era_int['longitude'], era_int['time']]).mean().to_xarray()
        era_int.to_netcdf(era_filepath +'/coarsened/'+ran_path.split('/')[-1])
        

In [53]:
year = '2014'
era_filepath = '/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/reanalysis_data/ERA5/regression_vars/'
u10 = xr.open_dataset(era_filepath + '10m_u_component_of_wind_'+ year+'.nc', chunks={'latitude':20, 'longitude':20, 'time':5})
v10 = xr.open_dataset(era_filepath + '10m_v_component_of_wind_'+ year+'.nc', chunks={'latitude':20, 'longitude':20, 'time':5})
sst = xr.open_dataset(era_filepath + 'sea_surface_temperature_'+ year+'.nc', chunks={'latitude':20, 'longitude':20, 'time':5})

In [8]:
dask.config.set({'array.slicing.split_large_chunks': True})

In [54]:
u10 = u10.coarsen({'latitude':10,'longitude':10},boundary='pad').mean()
v10 = v10.coarsen({'latitude':10,'longitude':10},boundary='pad').mean()
sst = sst.coarsen({'latitude':10,'longitude':10},boundary='pad').mean()

In [55]:
u10=u10.chunk(chunks='auto')
v10=v10.chunk(chunks='auto')
sst=sst.chunk(chunks='auto')

In [56]:
sst_lo = sst.differentiate('longitude')
sst_la = sst.differentiate('latitude')

In [22]:
Re= 6371000

In [45]:
colats = np.cos(sst_lo.latitude/180*np.pi)

In [67]:
Tadv =  - 1/(Re*colats) * sst_lo.sst * u10.u10

In [68]:
Tadv -= 1/Re * sst_la.sst * v10.v10

In [ ]:
Tadv.to_netcdf(era_filepath+'T_adv'+year+'.nc')

In [ ]:
Tadv